#### DNN기반 회귀 모델 구현 + 학습진행모니터링
- 데이터셋   : iris.csv
- 피쳐/속성  : 3개 Sepal_Length, Sepal-Width, Petal_Length
- 타겟/라벨  : 1개 Petal_Width
- 학습-방법  : 지도학습 > 회귀
- 알고리즘   : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성
- 프레임워크 : Pytorch
* * *
- 모니터링
    * 기준 : 검증데이터셋의loss또는score
    * 평가 : 학습데이터셋의loss또는score와 비교해서 학습 중단 여부를 결정
    * 선택 : 현제까지 진행된 모델의 파라미터(가중치,절편) 저장여부 또는 모델 전체 저장
* * *

[1] 모듈 로딩 및 데이터 준비

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from torchmetrics.regression import R2Score,MeanSquaredError
from torchinfo import summary
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [3]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수화 
def verCHeck():
    print(f'Pytorch.v {torch.__version__}')
    print(f'Pandas.v {pd.__version__}')
verCHeck()

Pytorch.v 2.4.1
Pandas.v 2.0.3


In [4]:
### 데이터 로딩
DATA_FILE='../data/iris.csv'
irisDF=pd.read_csv(DATA_FILE,usecols=[0,1,2,3])
irisDF.head()

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


[2] 모델 클레스 설계 및 정의
- 클레스목적 : iris 데이터를 학습 및 추론 목적
- 클레스이름 : IrisRegModel
- 부모클레스 : nn.Module
- 매개__변수 : 층별 입출력 개수 고정하기 떄문에 필요 없음!
- 속성__필드 : 
- 기능__역할 : _ _init_ _() : 모델 구조 설정, forward() : 순방향 학습 <= 오버라이딩(overriding)
- 클레스 구조
    * 입력층 : 입력 3개(피쳐)  출력 10개(퍼셉트론/뉴런 10개 존제)
    * 은닉층 : 입력 10개       출력 30개(퍼셉트론/뉴런 30개 존제)
    * 출력층 : 입력 30개       출력 1개(너비값)

- 활성화함수
    * 클레스 형태 ==> nn.MESLoss,nn.ReLU ==>_ _init_ _()함수
    * 함수 형태 ==> torch.nn.functional 아래

In [5]:
class IrisRegModel(nn.Module):
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.in_layer=nn.Linear(3,10)
        self.hd_layer=nn.Linear(10,30)
        self.out_layer=nn.Linear(30,1)

    #순방향 학습 진행 매서드
    def forward(self,input_data):
        # 입력층 
        y=F.relu(self.in_layer(input_data))
        # 은닉층 : 10개의 숫자 값(>=0)
        y=F.relu(self.hd_layer(y))
        # 출력층 : 30개의 숫자 값(>=0) 회귀이므로 바로 반환
        return self.out_layer(y)

In [6]:
## 모델 인스턴스 생성
model=IrisRegModel()
print(model)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [7]:
#모델 사용 메모리 정보 확인
summary(model,input_size=(100000,3))

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [100000, 1]               --
├─Linear: 1-1                            [100000, 10]              40
├─Linear: 1-2                            [100000, 30]              330
├─Linear: 1-3                            [100000, 1]               31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 40.10
Input size (MB): 1.20
Forward/backward pass size (MB): 32.80
Params size (MB): 0.00
Estimated Total Size (MB): 34.00

[3] 데이터셋 클레스 설계 및 정의
- 데이터__셋 : iris.csv
- 피쳐__개수 : 3개
- 타겟__개수 : 1개
- 클래스이름 : IrisDataset
- 부모클레스 : utils.data.Dataset
- 속성__필드 : featureDF,targetDF,n_rows,n_features
- 필수메서드
    * _ _init_ _(self) : 데이터셋 저장 및 전처리, 개발자가필요한 속성 설정
    * _ _len_ _(self)  : 데이터의 개수반환
    * _ _getitem_ _(self): 특정???피쳐와 타겟 반환

In [8]:
class IrisDataset(Dataset):
    
    def __init__(self,featureDF,targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
    
    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)
        # 피쳐와 타겟 반환
        return featureTS,targetTS

In [9]:
## 데이터셋 인스턴스 생성 
# DF에서 피쳐와 타겟 추출
featureDF=irisDF[irisDF.columns[:-1]]
targetDF=irisDF[irisDF.columns[-1:]]
# 커스텀 데이터셋 인스턴스 생성
irisDS=IrisDataset(featureDF,targetDF)

[4] 학습준비
- 학습횟수 : EPOCH 
- 배치크기 : BATCH_SIZE 
- 위치지정 : DEVICE 
- 학습률 : LR 가중치와 절편 업데이트 시 경사 하강법으로 업데이트 간격 설정 0.001~0.1

In [16]:
### 학습 진행 관련 설정
EPOCH=10
BATCH_SIZE=10
BACH_CNT=irisDF.shape[0]/BATCH_SIZE
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'
LR=0.001

- 인스턴스/객체 : 모델, 데이터셋, 최적화, (손실함수 ,성능지표)

In [17]:
# 모델 인스턴스
model=IrisRegModel()
# 데이터셋 인스턴스
X_train,X_test,y_train,y_test=train_test_split(featureDF,targetDF,random_state=1)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,random_state=1)
print(f'{X_train.shape},{X_test.shape},{X_val.shape}')
print(f'{y_train.shape},{y_test.shape},{y_val.shape}')
trainDS=IrisDataset(X_train,y_train)
valDS=IrisDataset(X_val,y_val)
testDS=IrisDataset(X_test,y_test)
# 데이터 로더 인스턴스
trainDL=DataLoader(trainDS,batch_size=BATCH_SIZE)

(84, 3),(38, 3),(28, 3)
(84, 1),(38, 1),(28, 1)


In [18]:
## [테스트]데이터로더
for feature,target in trainDL:
    print(feature.shape,target.shape)

torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([10, 3]) torch.Size([10, 1])
torch.Size([4, 3]) torch.Size([4, 1])


In [19]:
# 최적화 인스턴스 => W,b텐서 즉, model.paramiters()전달
optimizer=optim.Adam(model.parameters(),lr=LR)
# 손실함수 인스턴스 => 회귀, MSE,MAE,RMSE ...
reqLoss=nn.MSELoss()

[5]학습진행

In [20]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY,SCORE_HISTORY=[[],[]],[[],[]]
CNT=len(trainDL)

## 학습 모니터링/스캐줄링 설정 
# => LOSS_HISTORY,SCORE_HISTORY 활용
# => 임계기준 : 10번
BREAK_CNT=0
LIMIT_VALUE = 10 

for epoch in range(EPOCH):
    # 학습모드로 모델 설정
    model.train()
    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total,score_total=0,0
    for featureTS,targetTS in trainDL:
        # 학습진행
        pre_y=model(featureTS)
        # 손실계산
        loss=reqLoss(pre_y,targetTS)
        loss_total+=loss
        #성능평가 계산
        score=R2Score()(pre_y,targetTS)
        score_total+=score
        # 최적화진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # IF 에포크당 검증 한다면
    # 모델 검증모드
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)
        # 평가
        pre_val=model(val_featureTS)
        # 손실
        loss_val=reqLoss(pre_val,val_targetTS)
        # 성능평가
        score_val=R2Score()(pre_val,val_targetTS)
    
    # 에포크당 손실값과 성능 평가값 저장
    LOSS_HISTORY[0].append(loss_total/BATCH_SIZE)
    SCORE_HISTORY[0].append(score_total/BATCH_SIZE)
    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    # 학습 진행 모니터링/스캐줄링 - 검증 DS 기준
    # Loss OR Score 둘 중 하나 택 1
    # Loss기준
    if len(LOSS_HISTORY[0])>=2:
        if LOSS_HISTORY[1][-1] >= LOSS_HISTORY[1][-2]:BREAK_CNT+=1
    
    # Score기준
    # if len(SCORE_HISTORY[0])>=2:
    #     if SCORE_HISTORY[1][-1] <= SCORE_HISTORY[1][-2]:BREAK_CNT+=1
    
    # 학습 중단 여부 결정
    if BREAK_CNT>LIMIT_VALUE:
        print('성능 및 손실 개선이 없어서 학습 중단')

In [21]:
print(f'LOSS_HISTORY : {LOSS_HISTORY[0]}')
print(f'SCORE_HISTORY : {SCORE_HISTORY[0]}')

LOSS_HISTORY : [tensor(3.4532, grad_fn=<DivBackward0>), tensor(2.5264, grad_fn=<DivBackward0>), tensor(1.8868, grad_fn=<DivBackward0>), tensor(1.4131, grad_fn=<DivBackward0>), tensor(1.0360, grad_fn=<DivBackward0>), tensor(0.7382, grad_fn=<DivBackward0>), tensor(0.5124, grad_fn=<DivBackward0>), tensor(0.3577, grad_fn=<DivBackward0>), tensor(0.2696, grad_fn=<DivBackward0>), tensor(0.2282, grad_fn=<DivBackward0>)]
SCORE_HISTORY : [tensor(-6.5150, grad_fn=<DivBackward0>), tensor(-4.5325, grad_fn=<DivBackward0>), tensor(-3.1573, grad_fn=<DivBackward0>), tensor(-2.1426, grad_fn=<DivBackward0>), tensor(-1.3339, grad_fn=<DivBackward0>), tensor(-0.6915, grad_fn=<DivBackward0>), tensor(-0.2002, grad_fn=<DivBackward0>), tensor(0.1417, grad_fn=<DivBackward0>), tensor(0.3412, grad_fn=<DivBackward0>), tensor(0.4368, grad_fn=<DivBackward0>)]


- 모델 저장 방법
- 방법1 : 모델 파라미터만 저장
- 방법2 모델 설계 구조 및 파라미터까지 모두 저장

In [22]:
### 학습된 모델 파라미터 값 확인
model.state_dict()

OrderedDict([('in_layer.weight',
              tensor([[ 0.2403,  0.5714, -0.3053],
                      [-0.0578,  0.4003,  0.4421],
                      [-0.4849,  0.1960, -0.0106],
                      [-0.1423,  0.4512, -0.5267],
                      [ 0.3295,  0.4348,  0.3856],
                      [ 0.3270, -0.4480,  0.3895],
                      [ 0.3625, -0.3213,  0.4316],
                      [-0.2632, -0.0022,  0.2684],
                      [ 0.2481, -0.4552, -0.3676],
                      [ 0.2356,  0.3284, -0.2316]])),
             ('in_layer.bias',
              tensor([ 0.2537, -0.4799,  0.2504,  0.4054, -0.0306,  0.1120, -0.3036, -0.2650,
                      -0.1504, -0.3051])),
             ('hd_layer.weight',
              tensor([[-0.1266,  0.0596,  0.1455,  0.1971, -0.0957, -0.0125,  0.0678, -0.0548,
                        0.1368,  0.1904],
                      [ 0.2887,  0.3399,  0.2809,  0.0533,  0.0451,  0.0415,  0.0397, -0.0475,
                     

- [방법1]모델 파라미터 즉, 층별 가중치와 절편들

In [24]:
### models 폴더아레 프로젝트 폴더아레 모델 파일 저장
import os
# 저장경로
SAVE_PATH='../models/iris/'
# 저장파일명
SAVE_FILE='model_train_wbs.pth'

In [26]:
# 경로상 폴더 존제여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

In [28]:
#모델 저장
torch.save(model.state_dict(),SAVE_PATH+SAVE_FILE)

In [29]:
# 모델 즉, 가중치와 절편 로딩
# [1] 가중치와 절편 객체로 로딩
# [2] 모델의 state_dict

# 읽기
wbTS=torch.load(SAVE_PATH+SAVE_FILE,weights_only=True)
print(type(wbTS))

<class 'collections.OrderedDict'>


In [30]:
# 모델 인스턴스에 저장
model2=IrisRegModel()
model2.load_state_dict(wbTS)

<All keys matched successfully>

In [31]:
print(model2)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)
